In [44]:
### This program requires a local installation of APACHE SPARK to run
# Writing csv to Postgres/AWS Server

# import libraries
from pyspark import SparkFiles
from pyspark.sql.functions import to_date, to_timestamp
from getpass import getpass


# Start Spark Session
import findspark
findspark.init('C:\Spark\spark-3.2.2-bin-hadoop2.7')

# Build Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Qwest-Analytics-Dashboard-and-ML-Model").config("spark.jars", "C:\Spark\spark-3.2.2-bin-hadoop2.7\jars\postgresql-42.5.0.jar").getOrCreate()

In [45]:
#Import table into Spark Instance
resources_path = 'F:/Data Analytics/Final Project/Resources/'

cleaned_minute_aggregation_df = spark.read.csv(SparkFiles.get(f'{resources_path}cleaned_merged_minute_aggregate_data-2022-11-16.csv'), sep=',', header=True)
# Drop index column
cleaned_minute_aggregation_df = cleaned_minute_aggregation_df.drop('_c0')
cleaned_minute_aggregation_df.show()

+-------------------+---------+--------------+------------------------------+--------+-----------+------------------+-----------------+
|               time|  country|total_sessions|total_session_duration_seconds|operator|    channel|cleaned_content_id|            genre|
+-------------------+---------+--------------+------------------------------+--------+-----------+------------------+-----------------+
|2021-02-16 00:57:00|  Finland|           1.0|                          20.0|  Op_001|Channel_002|          PRO_1301|OPERA , CLASSICAL|
|2021-02-16 00:54:00|Australia|           4.0|                         240.0|  Op_002|Channel_002|          PRO_1301|OPERA , CLASSICAL|
|2021-02-16 00:55:00|Australia|           7.0|                         397.0|  Op_002|Channel_002|          PRO_1301|OPERA , CLASSICAL|
|2021-02-16 00:56:00|Australia|           8.0|                         388.0|  Op_002|Channel_002|          PRO_1301|OPERA , CLASSICAL|
|2021-02-16 00:57:00|Australia|           5.0|  

In [46]:
cleaned_minute_aggregation_df.printSchema()

root
 |-- time: string (nullable = true)
 |-- country: string (nullable = true)
 |-- total_sessions: string (nullable = true)
 |-- total_session_duration_seconds: string (nullable = true)
 |-- operator: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- cleaned_content_id: string (nullable = true)
 |-- genre: string (nullable = true)



In [49]:
# convert time column to datetime and total_sessions and total_sessions_duration_seconds to float
cleaned_minute_aggregation_df = cleaned_minute_aggregation_df.withColumn('time', to_timestamp('time'))
cleaned_minute_aggregation_df = cleaned_minute_aggregation_df.withColumn('total_sessions', cleaned_minute_aggregation_df.total_sessions.cast('double'))
cleaned_minute_aggregation_df = cleaned_minute_aggregation_df.withColumn('total_session_duration_seconds', cleaned_minute_aggregation_df.total_session_duration_seconds.cast('double'))

In [50]:
cleaned_minute_aggregation_df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |-- country: string (nullable = true)
 |-- total_sessions: double (nullable = true)
 |-- total_session_duration_seconds: double (nullable = true)
 |-- operator: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- cleaned_content_id: string (nullable = true)
 |-- genre: string (nullable = true)



In [51]:
# Connecting to database and exporting data

#Store environmental variables
password = getpass('Enter DataBase Password: ')

#Configure for RDS
mode = 'append'
jdbc_url="jdbc:postgresql://qwest-final-project.ccngkdwtiuvz.us-east-2.rds.amazonaws.com:5432/Qwest-Database"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [52]:
# Write Cleaned_Merged_Minute_Aggregation to table in RDS
cleaned_minute_aggregation_df.write.jdbc(url=jdbc_url, table = 'cleaned_merged_minute_aggregation', mode=mode, properties=config)

WRITE OTHER TABLES TO DATABASE

In [ ]:
# Write Cleaned_Advertising_Datav2.0.csv
### THIS CODE HAS NOT RUN ###

#Import table into Spark Instance
resources_path = 'F:\Data Analytics\Final Project\Qwest-Analytics-Dashboard-and-ML-Model\Resources'

cleaned_advertising_data = spark.read.csv(SparkFiles.get(f'{resources_path}Clean_Advertising_Datav2.0.csv'), sep=',', header=True)
# Drop index column
# cleaned_advertising_data = cleaned_minute_aggregation_df.drop('_c0')
cleaned_advertising_data.show()
